# Setup

In [10]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
from keras import ops, layers
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Define Hyperparameters

In [11]:
INP_SIZE = (4000, 3000)
TARGET_SIZE = (150, 150)
INTERPOLATION = 'bilinear'

AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 64
EPOCHS = 5

## Splitting the sample data

In [1]:
import os
from sklearn.model_selection import train_test_split
import shutil

# Define the root directory of your dataset
root_directory = r'C:\Users\msi1\Videos\Computer_Vision_Solution\Autonomous_Shoe_Spotter\sample_data'

# Define the subdirectories containing your classes
class_directories = [os.path.join(root_directory, d) for d in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory, d))]

# Initialize lists to store file paths for training and validation sets
train_files = []
val_files = []

# Split each class directory into train and validation sets
for class_dir in class_directories:
    # List all files in the current class directory
    files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
    
    # Split the files into train and validation sets
    train_files_class, val_files_class = train_test_split(files, test_size=0.3, random_state=42)  # Adjust test_size as needed
    
    # Append the split files to the respective lists
    train_files.extend(train_files_class)
    val_files.extend(val_files_class)

# Define directories for training and validation sets
train_directory = os.path.join(root_directory, 'training_data')
val_directory = os.path.join(root_directory, 'validation_data')

# Create train and validation directories if they don't exist
os.makedirs(train_directory, exist_ok=True)
os.makedirs(val_directory, exist_ok=True)

# Copy files to train and validation directories
for file in train_files:
    shutil.copy(file, os.path.join(train_directory, os.path.basename(file)))

for file in val_files:
    shutil.copy(file, os.path.join(val_directory, os.path.basename(file)))
